#### **Techniques for adversarial training**
In the end we trained on data augmentation, cropping and built a robust model.

In [ ]:
import torch
import sys
import os
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from onnxmodel.SecondModel import SecondModel

net = SecondModel(
    dataset_name="imagenet", batch_size=128, learning_rate=0.0003, num_epochs=30
)
model2_path = os.path.join(os.getcwd(), "final_second_model_imagenet.pth")
net.load_model(model2_path)
net2 = SecondModel(
    dataset_name="cifar10", batch_size=128, learning_rate=0.0003, num_epochs=50
)
model_path = os.path.join(os.getcwd(), "final_second_model_cifar10_60epochs.pth")
net2.load_model(model_path)


def test_clean_performance(model, test_loader):
    correct = 0
    total = 0
    model.net.eval()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(model.device), labels.to(model.device)
            outputs = model.net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def adversarial_training(model, train_loader, criterion, optimizer, epsilon=0.1):
    """
    Adversarial training procedure using FGSM.

    Parameters:
    - model: The neural network model to be trained.
    - train_loader: The data loader for the training dataset.
    - criterion: The loss function (e.g., CrossEntropyLoss).
    - optimizer: The optimization algorithm (e.g., Adam).
    - epsilon: The perturbation size for the FGSM attack.
    """
    model.train()  # Set the model in training mode

    for epoch in range(model.num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Step 1: Generate adversarial examples
            inputs.requires_grad = True  # Enable gradient computation for inputs
            optimizer.zero_grad()  # Clear previous gradients

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Step 2: Compute gradients of the loss with respect to the input
            loss.backward()  # Backpropagate the loss

            # Step 3: Generate adversarial perturbations
            perturbed_inputs = inputs + epsilon * inputs.grad.sign()

            # Step 4: Perform adversarial training (use perturbed inputs for the next forward pass)
            outputs_adv = model(perturbed_inputs)
            loss_adv = criterion(outputs_adv, labels)

            # Backpropagate adversarial loss
            optimizer.zero_grad()
            loss_adv.backward()
            optimizer.step()  # Update model weights

            print(
                f"Epoch {epoch + 1}, Loss: {loss.item():.4f}, Adversarial Loss: {loss_adv.item():.4f}"
            )

    print("Adversarial Training Completed!")